# Using Fire_MODIS_Thermal_Anomalies To Detect Fire Disk

## The fire detection strategy is based on absolute detection of a fire (when the fire strength is sufficient to detect), and on detection relative to its background (to account for variability of the surface temperature and reflection by sunlight).

Import Packages, update with prod_num, prod_name and TWC APIKEY

**################################################################################<br>
#Licensed Materials - Property of IBM <br>
#(C) Copyright IBM Corp. 2020,2021<br>
#US Government Users Restricted Rights - Use, duplication disclosure restricted<br>
#by GSA ADP Schedule Contract with IBM Corp.<br>
################################################################################**

In [1]:
!pip install arcgis

     |████████████████████████████████| 42.5 MB 5.8 MB/s eta 0:00:01███████████▏                    | 14.9 MB 5.8 MB/s eta 0:00:05
  Using cached gssapi-1.8.2-cp39-cp39-linux_x86_64.whl
  Using cached jupyter_client-6.1.12-py3-none-any.whl (112 kB)
  Using cached pandas-1.5.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.2 MB)
  Using cached keyring-23.13.1-py3-none-any.whl (37 kB)
  Using cached lerc-0.1.0-py3-none-any.whl
  Using cached ujson-5.7.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (52 kB)
  Using cached jupyterlab-3.5.2-py3-none-any.whl (8.8 MB)
  Using cached python_certifi_win32-1.6.1-py2.py3-none-any.whl (7.3 kB)
  Using cached pyshp-2.3.1-py2.py3-none-any.whl (46 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached requests_toolbelt-0.10.1-py2.py3-none-any.whl (54 kB)
  Using cached requests_gssapi-1.2.3-py2.py3-none-any.whl
  Using cached jaraco.classes-3.2.3-py3-none-any.whl (6.0 kB)
  Using cached jeepney-0.8.0-py3-n

     |████████████████████████████████| 52 kB 4.1 MB/s  eta 0:00:01
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
  Using cached jupyter_server-2.0.6-py3-none-any.whl (363 kB)
  Using cached nbclassic-0.4.8-py3-none-any.whl (9.8 MB)
  Using cached jupyterlab_server-2.18.0-py3-none-any.whl (56 kB)
  Using cached pyzmq-24.0.1-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Using cached jupyter_server_terminals-0.4.4-py3-none-any.whl (13 kB)
     |████████████████████████████████| 80 kB 21.1 MB/s eta 0:00:01
  Using cached nbformat-5.7.1-py3-none-any.whl (77 kB)
  Using cached jupyter_core-5.1.3-py3-none-any.whl (93 kB)
     |████████████████████████████████| 361 kB 67.9 MB/s eta 0:00:01
     |████████████████████████████████| 361 kB 90.3 MB/s eta 0:00:01
     |████████████████████████████████| 361 kB 91.5 MB/s eta 0:00:01
     |████████████████████████████████| 360 kB 48.8 MB/s eta 0:00:01
     |████████████████████████████████| 360 kB 88.5 MB/s eta 0:00:01
    

  Using cached notebook_shim-0.2.2-py3-none-any.whl (13 kB)
  Using cached setuptools_scm-7.1.0-py3-none-any.whl (43 kB)
  Created wheel for arcgis: filename=arcgis-2.1.0-cp39-cp39-linux_x86_64.whl size=7516548 sha256=f9c1fc8ccbec5e0dd98dc148790f7ecda3c3f7feff601d0c9e47d3cd99c2ec91
  Stored in directory: /tmp/1000650000/.cache/pip/wheels/8f/b2/25/a27b4cf00c32a4aa1ff01fa11a2eeb3fa52ad8783ddd0ee511
Successfully built arcgis
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 3.2.0
    Uninstalling jsonschema-3.2.0:
      Successfully uninstalled jsonschema-3.2.0
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.1.3
    Uninstalling nbformat-5.1.3:
      Successfully uninstalled nbformat-5.1.3
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 7.0.6
    Uninstalling jupyter-client-7.0.6:
      Successfully uninstalled jupyter-client-7.0.6
  Attempting uninstall: jinja2
    Found existing installation

In [2]:
######BL Snyder 10/11/19
# Fire_MODIS_Thermal_Anomalies
# MODIS Thermal Anomalies/Fire products are primarily derived from MODIS 4- and 11-micrometer radiances.

import requests,json,datetime, time # need to install these libraries to run
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureLayerCollection

##All TWC visual products have a product name and number
prod_num=str(603) # Radar Derived 24-Hour HailVision. 
APIKEY='f60f9ecb790749d38f9ecb790729d3be'  #This is IBM internal  key 

ModuleNotFoundError: No module named 'geomet.esri'

## Definition used to call for Product Inventrory

In [ ]:
########################PRODUCT INVENTORY CALL################################  
def InvCall():
    base1 = 'https://api.weather.com/v2/vector-api/products/'+prod_num+'/info?meta=true&max-times=12'
    base2 = '&apiKey='+APIKEY
    url = base1 + base2
    print(url)
    APIcall = requests.get(url) #Our API Call     to grab the latest forecast
    if APIcall.status_code == 200:  #if(APIcall.ok):
        data = APIcall.json()  # Just get most current run and next forecast
        t = data["products"][prod_num]["time"][0]        
        print (t)
        return t

## Definition used to call TWC API and Update ESRI

In [ ]:
def APIcall(t, ts): # function for pulling data 

    fc = {
        "type": "FeatureCollection",
        "features": []
    }
             
    base1 = 'https://api.weather.com/v2/vector-api/products/' + prod_num + '/features?time='+t 
    base2 = '&lod=0&x=0&y=0&tile-size=256&apiKey='+ APIKEY
    url = base1 + base2# make API URL   
    print (url)  
    data = requests.get(url).json() #Our API Call
    for y in data['features']:           
        f = {"type": "Feature", "properties": {}, "geometry": None}
        f['geometry'] = y['geometry']
        f['properties']['validTime'] = y['properties']['validTime']
        f['properties']['confidence'] = y['properties']['confidence']
        f['properties']['date'] = y['properties']['date']
        f['properties']['brightness'] = y['properties']['brightness']
        f['properties']['time'] = y['properties']['time']
        fc['features'].append(f)           
        
    with open("FireThermalAnomalies.geojson", "w") as outfile:
        json.dump(fc, outfile)
    ################################################################################
    ##  ADD CONTENT TO ARCGIS  
    # Connect to the GIS Using Your Credentials
    portalURL = "http://ibmmain.maps.arcgis.com"
    userID = "ESRI_USERNAME"
    password = "ESRI_PASSWORD"
    gis = GIS(portalURL, userID, password)
    #Update each piece of content
    existing_ID = '7dc6e8fa5d2f482b91550a90c2ba3f36'  # Map ID
    wx_item = gis.content.get(existing_ID)
    collection = features.FeatureLayerCollection.fromitem(wx_item)
    collection.manager.overwrite('FireThermalAnomalies.geojson')  
    print ("Updating Layer")


## Make the call to APICall with the latest date and time.

In [ ]:
t = InvCall()
print(t)
fmt = "%Y-%m-%d %H:%M:%S"
t_utc = datetime.datetime.utcfromtimestamp(int(t)/1000)
t_utc_formatted = t_utc.strftime(fmt)
APIcall(t, t_utc_formatted)